In [1]:
import tensorflow as tf
from ASR_Network import ASR_Network
from DataPipe import DataPipeFactory
from util_function import inform_pooling

/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/p

In [2]:
# Disable all GPUS
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [3]:
ds = DataPipeFactory('../DataFolder/Tensorflow_DataRecord/Student_Answer_Record.tfrecord',
                     '../DataFolder/Siri_Related/Siri_Reference_Sample',
                     '../DataFolder/Siri_Related/Siri_Dense_Index', cache='../DataFolder/cache/datapipe/cached')
# dsp = ds.get_batch_data(10)
# it = iter(dsp)
ds.get_raw_data()

Load cache from ../DataFolder/cache/datapipe/cached


<_LoadDataset element_spec={'ref_mfcc': TensorSpec(shape=(4, None, 80), dtype=tf.float32, name=None), 'MatchSegment': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'valid_ref_word': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'MatchReference': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'valid_ref_start': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'RecordName': TensorSpec(shape=(), dtype=tf.string, name=None), 'passage_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'valid_ref_duration': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'valid_stu_duration': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'valid_stu_start': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'stu_mfcc': TensorSpec(shape=(None, 80), dtype=tf.float32, name=None)}>

In [4]:
base_feature = ([None, 80], 512, [128, 256, 384, 512], 3, 2)
base_feature_name = ('input_shape', 'feature_depth', 'channels_list', 'filter_size', 'stack_size')
dense_feature = ([1280], 512, [512, 512], 2)
dense_feature_name = ('input_shape', 'output_shape', 'channels_list', 'stack_size')
word_prediction = ([512], 2500, [512, 1024], 2)
word_prediction_name = ('input_shape', 'output_shape', 'channels_list', 'stack_size')
# Generate dict using zip
# base_feature = dict(zip(base_feature_name, base_feature))
# dense_feature = dict(zip(dense_feature_name, dense_feature))
# word_prediction = dict(zip(word_prediction_name, word_prediction))
base_ratio = 125
hyper_para = {'base_feature': dict(zip(base_feature_name, base_feature)),
              'dense_feature': dict(zip(dense_feature_name, dense_feature)),
              'word_prediction': dict(zip(word_prediction_name, word_prediction)),
              'base_ratio': base_ratio,
              'batch_num': 10}
C = ASR_Network(**hyper_para)

In [5]:
def unpack(d):
    value_s = d['stu_mfcc']
    start_s = tf.RaggedTensor.from_tensor(d['valid_stu_start'], padding=-1.)
    duration_s = tf.RaggedTensor.from_tensor(d['valid_stu_duration'], padding=-1.)

    #unpack with another key ref_mfcc, valid_ref_start, valid_ref_duration
    value_f = d['ref_mfcc']
    start_f = tf.RaggedTensor.from_tensor(d['valid_ref_start'], padding=-1.)
    duration_f = tf.RaggedTensor.from_tensor(d['valid_ref_duration'], padding=-1.)

    # unpack valid_ref_word

    words = tf.RaggedTensor.from_tensor(d['valid_ref_word'], padding=-1)
    return ((value_s, (start_s, duration_s)), (value_f, (start_f, duration_f))), words

In [6]:
dst, dse = ds.get_batch_data_with_eval(10, 4096, addition_map=unpack)

In [7]:
dst

<_PrefetchDataset element_spec=(((TensorSpec(shape=(10, None, 80), dtype=tf.float32, name=None), (RaggedTensorSpec(TensorShape([10, None]), tf.float32, 1, tf.int64), RaggedTensorSpec(TensorShape([10, None]), tf.float32, 1, tf.int64))), (TensorSpec(shape=(10, None, 80), dtype=tf.float32, name=None), (RaggedTensorSpec(TensorShape([10, None]), tf.float32, 1, tf.int64), RaggedTensorSpec(TensorShape([10, None]), tf.float32, 1, tf.int64)))), RaggedTensorSpec(TensorShape([10, None]), tf.int64, 1, tf.int64))>

In [8]:
it = iter(dst)

2023-04-17 00:16:12.819268: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [9]:
data = next(it)

In [ ]:
def call(self, inputs, training=False, mask=None):
    audio, (start, duration) = inputs
    # compute the base network
    base_output, maps = self.base_network(audio, training=training)
    # combine base output and maps
    total_maps = [base_output] + maps
    # pooling the total maps
    pooled_maps = self.pooling(total_maps, start, duration)
    # compute the deep feature
    deep_feature = self.deep_feature(pooled_maps, training=training)
    # compute the word prediction
    word_prediction = self.word_prediction(deep_feature, training=training)
    return word_prediction, deep_feature

In [ ]:
C(data[0][0])

In [ ]:
C.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
C.fit(dst, epochs=1)

In [ ]:
nl = tf.keras.layers.BatchNormalization()

In [ ]:
Ragg1 = tf.RaggedTensor.from_row_splits(tf.random.normal([100,80]), row_splits=tf.range(0, 101, 5))
Ragg2 = tf.RaggedTensor.from_row_splits(tf.random.normal([100,80]), row_splits=tf.range(0, 101, 10))

In [ ]:
tf.ragged.map_flat_values(nl, Ragg2)

In [ ]:
class RaggedBatchNormalization(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.batch_norm = tf.keras.layers.BatchNormalization(**kwargs)

    def call(self, inputs, training=None, mask=None):
        # Pass `training` and `mask` arguments to the `call` method of the BatchNormalization layer
        return tf.ragged.map_flat_values(
            lambda x: self.batch_norm(x, training=training, mask=mask), inputs
        )

    def get_config(self):
        return super().get_config()

    @classmethod
    def from_config(cls, config):
        return super().from_config(config)

In [ ]:
rnl = RaggedBatchNormalization()

In [ ]:
rnl(Ragg1)

In [ ]:
nl(tf.random.normal([100,80]))

In [ ]:
Ragg1.ragged_rank

In [ ]:
Ragg1.flat_values

In [ ]:
C.evaluate(dse)

In [ ]:
data

In [ ]:
dst

In [ ]:
dst = ds.get_batch_data(10)

In [ ]:
dst

In [ ]:
dst, dse = ds.get_batch_data_with_eval(10, 4096, addition_map=unpack)

In [ ]:
dst

In [10]:
data

(((<tf.Tensor: shape=(10, 6683, 80), dtype=float32, numpy=
   array([[[-0.24186735, -0.86503196, -1.5029379 , ..., -2.2397168 ,
            -2.5748942 , -2.915472  ],
           [-0.47064173, -1.0578283 , -1.4292537 , ..., -2.3496437 ,
            -2.6487715 , -2.861642  ],
           [-0.8759077 , -1.4377421 , -1.360072  , ..., -2.4746628 ,
            -2.6505232 , -2.7826545 ],
           ...,
           [-1.        , -1.        , -1.        , ..., -1.        ,
            -1.        , -1.        ],
           [-1.        , -1.        , -1.        , ..., -1.        ,
            -1.        , -1.        ],
           [-1.        , -1.        , -1.        , ..., -1.        ,
            -1.        , -1.        ]],
   
          [[-1.2407596 , -0.77480745, -0.58651614, ..., -0.70419216,
            -1.0959346 , -0.98496103],
           [-0.91370475, -0.6822565 , -0.73391676, ..., -0.9046972 ,
            -1.0849228 , -1.0065882 ],
           [-0.73431945, -0.60122186, -0.63412577, ..., 